In [2]:
import numpy as np
import copy
from time import perf_counter
from collections import Counter, deque
from functools import lru_cache
import numba
from itertools import permutations, combinations, product

In [60]:
f = open("input.txt", "r")
rawstring = f.read()
f.close()

In [61]:
class Field():
    def __init__(self, map) -> None:
        self.map = map
        self.homes = {
            'A': [[2, 3], [3, 3]],
            'B': [[2, 5], [3, 5]],
            'C': [[2, 7], [3, 7]],
            'D': [[2, 9], [3, 9]]
        }
        self.taboos = [[1, 3], [1, 5], [1, 7], [1, 9]]
        self.movable_pos = [[1, 1], [1, 2], [1, 4],
                            [1, 6], [1, 8], [1, 10], [1, 11]]
        self.costs = {'A': 1, 'B': 10, 'C': 100, 'D': 1000}

    def getPossibleMoves(self, playerPos, c, idx, cost):
        coord = playerPos[c][idx]
        otherFigs = []
        for color in playerPos.keys():
            for i in playerPos[color]:
                if i != idx and color != c:
                    otherFigs.append(i)

        # Todo: check if in home:
        if list(coord) == self.homes[c][1]:
            return cost

        if list(coord) in self.homes[c] and list(playerPos[c][0 if idx else 1]) in self.homes[c]:
            return cost
            print(f"color {c} correct")
        #
        if coord[0] == 3:
            # have to check whether field above is free
            if (coord[0]-1, coord[1]) in otherFigs:
                return

        otherFigs_target = [[x, y] for x, y in otherFigs if x == 1]
        otherFigs_target_y = sorted([y for x, y in otherFigs_target])
        next_left = next(
            (y for y in otherFigs_target_y if (y < coord[1])), -100)
        next_right = next((y for y in reversed(
            otherFigs_target_y) if (y > coord[1])), 100)

        homeEmpty = True
        colorFalse = False
        lowerCorrect = False
        upperEmpty = True
        for color in playerPos.keys():
            for i in playerPos[color]:
                i = list(i)
                if i in self.homes[c]:
                    if color != c:
                        colorFalse = True
                    elif i == self.homes[c][1] and color == c:
                        lowerCorrect = True
                    else:
                        upperEmpty = False
                    homeEmpty = False

        homeReachable = False
        if next_left < self.homes[c][0][1] < next_right:
            homeReachable = True

        if homeEmpty:
            # move into bottom slot
            retVal = []
            p = copy.deepcopy(playerPos)
            p[c][idx] = self.homes[c][1]
            retVal.append((p, (
                cost + self.costs[c]*(abs(coord[0]-p[c]
                                      [idx][0])+abs(coord[1]-p[c][idx][1]))
            )))
            return retVal

        if lowerCorrect and upperEmpty:
            # move into top slot
            retVal = []
            p = copy.deepcopy(playerPos)
            p[c][idx] = self.homes[c][0]
            retVal.append((p, (
                cost + self.costs[c]*(abs(coord[0]-p[c]
                                      [idx][0])+abs(coord[1]-p[c][idx][1]))
            )))
            return retVal

        
        new_pos = [i for i in self.movable_pos if next_left < i[1] < next_right]

        retVal = []
        for i in new_pos:
            p = copy.deepcopy(playerPos)
            p[c][idx] = i
            retVal.append((p, (
                cost + self.costs[c]*(abs(coord[0]-p[c]
                                      [idx][0])+abs(coord[1]-p[c][idx][1]))
            )))
        return retVal

    def print(self, playerPos):
        temp = copy.deepcopy(self.map)
        for color in playerPos.keys():
            for i in playerPos[color]:
                temp[i[0], i[1]] = color

        for i in range(temp.shape[0]):
            print("".join(temp[i, :]))
        pass

    @staticmethod
    def fromString(string):
        map = np.array([np.array(list(line.ljust(len(string.splitlines()[0]))))
                       for line in rawstring.splitlines()])

        playerPos = dict()
        for i in 'ABCD':
            playerPos[i] = [(x, y) for x, y in zip(
                np.where(map == i)[0], np.where(map == i)[1])]

        for i in [' ']:
            map[map == i] = '#'
        for i in "ABCD":
            map[map == i] = '.'
        return Field(map), playerPos


In [62]:
m,p = Field.fromString(rawstring)

In [63]:
p_2 = m.getPossibleMoves(p,'A',0,0)[0]

TypeError: 'NoneType' object is not subscriptable

In [64]:
p_2[0]

{'A': [[2, 3], (3, 3)],
 'B': [(2, 9), (3, 5)],
 'C': [(1, 1), (3, 9)],
 'D': [(2, 7), (3, 7)]}

In [65]:

m.getPossibleMoves(p_2[0],'A',1,0)

0

In [66]:
m.print(p_1[0][0])

NameError: name 'p_1' is not defined

In [67]:
m.getPossibleMoves(p_1[0][0],'A',0,0)

NameError: name 'p_1' is not defined

In [68]:
import heapq

In [69]:
heap = []
heapq.heappush(heap, (0,'one', 1))
heapq.heappush(heap, (12,'one', 1))
heapq.heappush(heap, (-12,'one', 1))

In [70]:
heapq.heappop(heap)

(-12, 'one', 1)

In [71]:
heap.popleft()

AttributeError: 'list' object has no attribute 'popleft'

In [72]:
openConfigs = [(0,0,p,0)]
count = 0
while len(openConfigs) != 0:
    cConfig = heapq.heappop(openConfigs)
    count += 1
    if (count % 1000) == 0:
        m.print(cConfig[2])
    int_rets = 0
    tempList = []
    for c in 'ABCD':
        for i in [0,1]:
            temp = m.getPossibleMoves(cConfig[2],c,i,cConfig[3])
            if type(temp) == np.int64 or type(temp) == int:
                int_rets += 1
            elif temp != None:
                for newConfig in temp:
                    tempList.append(newConfig)
    for t in tempList:
        heapq.heappush(openConfigs, (8-int_rets,id(t[0]),t[0],t[1]))
        #openConfigs.append((int_rets,t[0],t[1]))
    if int_rets == 8:
        print(f"solution found: {temp}")

solution found: 10639
solution found: 10639
solution found: 10639
solution found: 10639
solution found: 10639
solution found: 10639
solution found: 11439
solution found: 11439
solution found: 11439
solution found: 11439
solution found: 11439
solution found: 11439
solution found: 11239
solution found: 11239
solution found: 11239
solution found: 11239
solution found: 11439
solution found: 11439
solution found: 11439
solution found: 11439
solution found: 12239
solution found: 12239
solution found: 11439
solution found: 11439
solution found: 11639
solution found: 11639
solution found: 12839
solution found: 12839
solution found: 13439
solution found: 13439
solution found: 13639
solution found: 13639
solution found: 13439
solution found: 13439
solution found: 14639
solution found: 14639
solution found: 13439
solution found: 13439
solution found: 13639
solution found: 13639
solution found: 14639
solution found: 14639
solution found: 14639
solution found: 14639
solution found: 14639
solution f

KeyboardInterrupt: 

In [93]:
l = deque([1,2,3])

In [96]:
l.popleft()

1